In [1]:
import pandas as pd
from bs4 import BeautifulSoup 

In [ ]:
with open(r"...path_to\RDsqr-KG\Datasets\Orphanet\en_product1.xml", "r", encoding="ISO-8859-1") as file:
    soup1 = BeautifulSoup(file, "xml")  #disease_vocabulary

with open(r"...path_to\RDsqr-KG\Datasets\Orphanet\en_product4.xml", "r", encoding="ISO-8859-1") as file:
    soup2 = BeautifulSoup(file, "xml")  #disease_phenotype

with open(r"...path_to\RDsqr-KG\Datasets\Orphanet\en_product6.xml", "r", encoding="ISO-8859-1") as file:
    soup3 = BeautifulSoup(file, "xml")  #disease_gene

<>:3: SyntaxWarning: invalid escape sequence '\O'
<>:3: SyntaxWarning: invalid escape sequence '\O'
C:\Users\Arun\AppData\Local\Temp\ipykernel_28808\1938198200.py:3: SyntaxWarning: invalid escape sequence '\O'
  """


'\nwith open(r"...path_to\\Orphanet\\en_product4.xml", "r", encoding="ISO-8859-1") as file:\n    soup2 = BeautifulSoup(file, "xml")  #disease_phenotype\n\nwith open(r"...path_to\\Orphanet\\en_product6.xml", "r", encoding="ISO-8859-1") as file:\n    soup3 = BeautifulSoup(file, "xml")  #disease_gene'

In [3]:
disease_vocabulary = []
disease_phenotype = []
disease_gene = []

In [4]:
for disorder in soup1.find_all("Disorder"):
            disorder_code = disorder.find("OrphaCode").text if disorder.find("OrphaCode") else "N/A"
            disorder_name = disorder.find("Name").text if disorder.find("Name") else "N/A"

            external_references = {}
            external_reference_list = disorder.find("ExternalReferenceList")
            if external_reference_list:
                for reference in external_reference_list.find_all("ExternalReference"):
                    source = reference.find("Source").text if reference.find("Source") else "N/A"
                    ref = reference.find("Reference").text if reference.find("Reference") else "N/A"
                    external_references[source] = ref

            disease_vocabulary.append((disorder_code, disorder_name, external_references))

In [5]:
df_disease_vocab = pd.DataFrame(disease_vocabulary, columns=['OrphaCode', 'DisorderName', 'ExternalReferences'])

external_disease_ref_df = df_disease_vocab['ExternalReferences'].apply(pd.Series).fillna("N/A") # references into different columns

df_disease_vocab = pd.concat([df_disease_vocab.drop(columns=['ExternalReferences']), external_disease_ref_df], axis=1) #add references to dataframe

In [6]:
df_disease_vocab['DisorderName'] = df_disease_vocab['DisorderName'].str.replace(r'^NON RARE IN EUROPE:\s*', '', regex=True).str.strip()
df_disease_vocab = df_disease_vocab[~df_disease_vocab['DisorderName'].str.startswith('OBSOLETE')]

In [9]:
for hpo_set_status in soup2.find_all("HPODisorderSetStatus"):
            disorder = hpo_set_status.find("Disorder")
            disorder_code = disorder.find("OrphaCode").text
            disorder_name = disorder.find("Name").text

            for association in hpo_set_status.find_all("HPODisorderAssociation"):
                hpo_id = association.find("HPOId").text
                hpo_term = association.find("HPOTerm").text

                disease_phenotype.append((disorder_code, disorder_name, hpo_id, hpo_term))

In [10]:
df_disease_phenotype = pd.DataFrame(disease_phenotype, columns=['OrphaCode', 'DisorderName', 'HPOId', 'HPOTerm'])

In [11]:
for disorder in soup3.find_all("Disorder"):
            disorder_code = disorder.find("OrphaCode").text if disorder.find("OrphaCode") else "N/A"
            disorder_name = disorder.find("Name").text if disorder.find("Name") else "N/A"

            for association in disorder.find_all("DisorderGeneAssociation"):
                gene = association.find("Gene")
                gene_name = gene.find("Name").text if gene and gene.find("Name") else "N/A"
                gene_symbol = gene.find("Symbol").text if gene and gene.find("Symbol") else "N/A"

                external_references = {}
                external_reference_list = association.find("ExternalReferenceList")
                if external_reference_list:
                    for reference in external_reference_list.find_all("ExternalReference"):
                        source = reference.find("Source").text
                        ref = reference.find("Reference").text
                        external_references[source] = ref

                disease_gene.append((disorder_code, disorder_name, gene_name, gene_symbol, external_references))

In [12]:
df_disease_gene = pd.DataFrame(disease_gene, columns=['OrphaCode', 'DisorderName', 'GeneName', 'GeneSymbol', 'ExternalReferences'])

external_gene_ref_df = df_disease_gene['ExternalReferences'].apply(pd.Series).fillna("N/A") # references into different columns

df_disease_gene = pd.concat([df_disease_gene.drop(columns=['ExternalReferences']), external_gene_ref_df], axis=1) #add references to dataframe

In [7]:
df_disease_vocab.shape

(9946, 9)

In [14]:
df_disease_phenotype.shape

(112599, 4)

In [15]:
df_disease_gene.shape

(8351, 11)

In [ ]:
df_disease_vocab.to_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\disease_references.csv", index=False)
df_disease_phenotype.to_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\disease_phenotype_orphanet.csv", index=False)
df_disease_gene.to_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\disease_gene_orphanet.csv", index=False)